# LR/CNN baseline training + evaluation

In [1]:
import numpy as np
import torch
import os, sys

In [2]:
project_root = os.path.abspath("..")  # mini-project root
src_root = os.path.join(project_root, "src")

# 1) Remove any existing occurrences
sys.path = [p for p in sys.path if p not in (project_root, src_root)]

# 2) Insert project paths at the very front
sys.path.insert(0, project_root)
sys.path.insert(1, src_root)  # not strictly needed, but fine

print("CWD:", os.getcwd())
print("sys.path first entries:")
print("\n".join(sys.path[:5]))

if "src" in sys.modules:
    del sys.modules["src"]

CWD: /Users/yujincha/Documents/snu_ms/coursework/2025-2/M1522.003300 모바일 및 유비쿼터스 컴퓨팅 (이영기, Graduate)/mini-project/notebooks
sys.path first entries:
/Users/yujincha/Documents/snu_ms/coursework/2025-2/M1522.003300 모바일 및 유비쿼터스 컴퓨팅 (이영기, Graduate)/mini-project
/Users/yujincha/Documents/snu_ms/coursework/2025-2/M1522.003300 모바일 및 유비쿼터스 컴퓨팅 (이영기, Graduate)/mini-project/src
/Users/yujincha/miniforge3/envs/har-ldp/lib/python310.zip
/Users/yujincha/miniforge3/envs/har-ldp/lib/python3.10
/Users/yujincha/miniforge3/envs/har-ldp/lib/python3.10/lib-dynload


In [3]:
import src
print("src.__file__:", src.__file__)

src.__file__: /Users/yujincha/Documents/snu_ms/coursework/2025-2/M1522.003300 모바일 및 유비쿼터스 컴퓨팅 (이영기, Graduate)/mini-project/src/__init__.py


In [4]:
from src.utils import set_seed, get_device
from src.data import load_har, make_train_val_test, make_dataloaders
from src.models import LRBaseline, SmallCNN
from src.train import train_classifier, evaluate_classifier

In [5]:
def run_lr_baseline(data_root="./data"):
    set_seed(42)
    device = get_device()
    print("Device:", device)

    X_train, y_train, X_test, y_test = load_har(data_root)
    (X_tr, y_tr), (X_val, y_val), (X_te, y_te) = make_train_val_test(
        X_train, y_train, X_test, y_test, val_ratio=0.1, seed=42
    )

    train_loader, val_loader, test_loader = make_dataloaders(
        X_tr, y_tr, X_val, y_val, X_te, y_te, batch_size=128
    )

    input_dim = X_tr.shape[1]
    num_classes = int(y_train.max() + 1)

    model = LRBaseline(input_dim=input_dim, num_classes=num_classes)
    history = train_classifier(
        model, train_loader, val_loader, device,
        epochs=20, lr=1e-3
    )

    test_loss, test_acc = evaluate_classifier(model, test_loader, device)
    print(f"[LR] Test loss={test_loss:.4f}, Test acc={test_acc:.4f}")


def run_cnn_baseline(data_root="../data"):
    set_seed(42)
    device = get_device()
    print("Device:", device)

    X_train, y_train, X_test, y_test = load_har(data_root)
    (X_tr, y_tr), (X_val, y_val), (X_te, y_te) = make_train_val_test(
        X_train, y_train, X_test, y_test, val_ratio=0.1, seed=42
    )

    train_loader, val_loader, test_loader = make_dataloaders(
        X_tr, y_tr, X_val, y_val, X_te, y_te, batch_size=128
    )

    # Inspect one batch to confirm shape
    sample_X, _ = next(iter(train_loader))
    print("Sample batch shape:", sample_X.shape)   # should be (B, 1, 561)
    in_channels = sample_X.shape[1]
    num_classes = int(y_train.max() + 1)

    model = SmallCNN(in_channels=in_channels, num_classes=num_classes)

    history = train_classifier(
        model, train_loader, val_loader, device,
        epochs=20, lr=1e-3
    )

    test_loss, test_acc = evaluate_classifier(model, test_loader, device)
    print(f"[CNN] Test loss={test_loss:.4f}, Test acc={test_acc:.4f}")
    return history, (test_loss, test_acc)


cnn_hist, cnn_metrics = run_cnn_baseline()

Device: cpu
dataset_dir: ../data/UCI_HAR_Dataset
Sample batch shape: torch.Size([128, 1, 561])
Epoch 01: train_loss=1.4712, train_acc=0.3038, val_loss=1.0958, val_acc=0.3932
Epoch 02: train_loss=1.0835, train_acc=0.4064, val_loss=1.0617, val_acc=0.4014
Epoch 03: train_loss=1.0661, train_acc=0.4138, val_loss=1.0559, val_acc=0.3918
Epoch 04: train_loss=1.0681, train_acc=0.4076, val_loss=1.0535, val_acc=0.3932
Epoch 05: train_loss=1.0543, train_acc=0.4333, val_loss=1.0686, val_acc=0.3728
Epoch 06: train_loss=1.0470, train_acc=0.4381, val_loss=1.0386, val_acc=0.4653
Epoch 07: train_loss=1.0334, train_acc=0.4576, val_loss=1.0336, val_acc=0.4735
Epoch 08: train_loss=1.0084, train_acc=0.5040, val_loss=0.9885, val_acc=0.4952
Epoch 09: train_loss=0.9778, train_acc=0.5330, val_loss=0.9437, val_acc=0.5673
Epoch 10: train_loss=0.9177, train_acc=0.5782, val_loss=0.9105, val_acc=0.5034
Epoch 11: train_loss=0.8645, train_acc=0.6050, val_loss=0.8234, val_acc=0.6259
Epoch 12: train_loss=0.7926, train_a